In [1]:
import os

In [2]:
%pwd

'd:\\ML_MLOPS_END_project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\ML_MLOPS_END_project'

In [5]:
# entity

from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_url: str 
    local_data_file: Path 
    unzip_dir: Path

In [6]:
# constants
from pathlib import Path 
config_file_path = Path("config/config.yaml")
params_file_path = Path("params.yaml")
schema_file_path = Path("schema.yaml")

In [7]:
# configurationmanager
from mlProject.constants import*
from mlProject.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=config_file_path,
        params_filepath=params_file_path,
        schema_filepath=schema_file_path):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [8]:
# components

import os
import urllib.request as request 
import zipfile
from mlProject import logger 
from mlProject.utils.common import get_size

In [9]:
# componenst 2
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config 
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_url,
                filename =self.config.local_data_file
            )
            logger.info(f"{filename} download with following into :\{headers}")
        else:
            logger.info(f"file alredy exist of size:{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
#pipeline

try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config() # cm2ndpart
    data_ingestion=DataIngestion(config=data_ingestion_config) # component 
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-10 14:48:42,777: INFO: common: yaml file: config\config.yaml loaded succesfuly]
[2024-02-10 14:48:42,782: INFO: common: yaml file: params.yaml loaded succesfuly]
[2024-02-10 14:48:42,794: INFO: common: yaml file: schema.yaml loaded succesfuly]
[2024-02-10 14:48:42,799: INFO: common: created directory at :artifacts]
[2024-02-10 14:48:42,801: INFO: common: created directory at :artifacts/data_ingestion]
[2024-02-10 14:48:45,342: INFO: 1941492368: artifacts/data_ingestion/data.zip download with following into :\Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: ACF2:6687:4BE7CA:5B4FED:65C73F74
Accept-Ranges: bytes
Date: Sat, 10 Feb 2024